# 加载和查询 Obsidian 笔记

结论：

- 能查，看起来准确性没问题
- 不能回溯到具体的文档
- 加载时已经做了分割，而不是按照文档为单位

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 2.15 s, sys: 232 ms, total: 2.38 s
Wall time: 2 s


In [2]:
%%time
%%capture

!pip install llama_index.readers.obsidian

CPU times: user 8.53 ms, sys: 8.02 ms, total: 16.6 ms
Wall time: 1.93 s


## 加载数据

In [3]:
%%time

from llama_index.readers.obsidian import ObsidianReader


documents = ObsidianReader(
    "/models/obsidian/kb/知识库"
).load_data() 

CPU times: user 157 ms, sys: 15.2 ms, total: 172 ms
Wall time: 173 ms


In [4]:
len(documents)

21

## 创建索引

In [5]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents
)

CPU times: user 247 ms, sys: 41.4 ms, total: 289 ms
Wall time: 3.26 s


## 查询

In [6]:
%%time

query_engine = index.as_query_engine(
    streaming=True
)
streaming_response = query_engine.query("docker容器在后台执行的命令")
streaming_response.print_response_stream()

docker 容器在后台执行的命令通常使用 `docker run` 命令，并通过添加 `-d` 参数来指定。根据提供的上下文信息，以下是在后台执行的命令示例：

1. 使用基础命令行方式：
   ```bash
   docker run -d -t --rm bash
   ```
   
2. 在 Docker Compose 文件中定义服务时：
   ```yaml
   services:
     bash:
       container_name: bash
       image: bash:latest
       tty: true
   ```

3. 使用 `docker-compose.yaml` 文件启动容器时，可以指定用户 UID 和组 ID 以及其他配置选项。例如：

   ```bash
   CURRENT_UID=$(id -u):$(id -g) docker-compose up -d
   ```
   
   这里使用了 `CURRENT_UID` 变量来启动容器，并通过 `-d` 参数在后台运行。

这些命令和配置都是为了在 Docker 容器中执行指定的命令或服务，且通常会设置一些额外参数以满足特定需求。CPU times: user 251 ms, sys: 33.8 ms, total: 285 ms
Wall time: 6.42 s


In [7]:
%%time

streaming_response = query_engine.query("重启docker的命令")
streaming_response.print_response_stream()

要重启Docker，可以使用以下命令：
```properties
sudo systemctl daemon-reload && sudo systemctl restart docker
```CPU times: user 72.7 ms, sys: 7.54 ms, total: 80.3 ms
Wall time: 862 ms


In [8]:
%%time

streaming_response = query_engine.query("怎么注册微信小程序")
streaming_response.print_response_stream()

要注册微信小程序，请按照以下步骤操作：

1. 访问微信小程序的注册地址：[https://mp.weixin.qq.com/](https://mp.weixin.qq.com/)。
2. 创建一个单独的邮箱，并使用这个邮箱进行注册。在注册过程中，可能需要通过验证邮件来确认邮箱的有效性。
3. 完成微信小程序的基本信息填写和审核流程。

请注意，在完成上述步骤后，您还需要安装“微信开发者工具”，创建原型项目，并进行原生小程序开发。CPU times: user 155 ms, sys: 12.3 ms, total: 167 ms
Wall time: 2.35 s


In [9]:
%%time

streaming_response = query_engine.query("原生小程序怎么开发")
streaming_response.print_response_stream()

进行原生小程序开发有以下两种方式：

1. **基于小程序原生API和组件API实现项目**：
   - 这是最直接的方法，直接使用原生API来开发应用。
   - 性能表现最佳，因为是直接利用底层的原生能力。
   - 但这种方法不适用于开发复杂的应用，也不利于代码复用。

2. **使用第三方开发框架**（例如Taro等）：
   - 这种方式最终会将代码编译为小程序的原生形式。
   - 性能可能不如直接使用原生API，因为需要通过框架进行转换。
   - 优点是有利于代码复用，比如可以将同一份代码用于开发支付宝小程序等其他平台的应用。
   - 另外，使用第三方框架有助于自动化开发和部署过程。CPU times: user 201 ms, sys: 24.7 ms, total: 226 ms
Wall time: 3.74 s


In [10]:
streaming_response

StreamingResponse(response_gen=<generator object stream_chat_response_to_tokens.<locals>.gen at 0x7f48a5068970>, source_nodes=[NodeWithScore(node=TextNode(id_='6df3656c-c87e-49bf-853c-973173a82151', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f58d6f6b-2ba0-4745-bb8d-5fb6b9c02c00', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ea4b00b3713acc970aa70f64a0682042db72fc3a3218bdba61bcb1e949bc1c95')}, text='主要步骤\n\n- 注册微信小程序\n- 安装 “微信开发者工具”\n- 在 “微信开发者工具” 中创建原型项目\n- 进行原生小程序开发', mimetype='text/plain', start_char_idx=2, end_char_idx=66, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7927414393375357), NodeWithScore(node=TextNode(id_='19e4a003-457f-4e4b-8616-8f90ef73efa0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOU

In [11]:
len(streaming_response.source_nodes)

2

In [12]:
streaming_response.source_nodes[0]

NodeWithScore(node=TextNode(id_='6df3656c-c87e-49bf-853c-973173a82151', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f58d6f6b-2ba0-4745-bb8d-5fb6b9c02c00', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ea4b00b3713acc970aa70f64a0682042db72fc3a3218bdba61bcb1e949bc1c95')}, text='主要步骤\n\n- 注册微信小程序\n- 安装 “微信开发者工具”\n- 在 “微信开发者工具” 中创建原型项目\n- 进行原生小程序开发', mimetype='text/plain', start_char_idx=2, end_char_idx=66, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7927414393375357)

In [13]:
streaming_response.source_nodes[1]

NodeWithScore(node=TextNode(id_='19e4a003-457f-4e4b-8616-8f90ef73efa0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2bf10967-d7a5-4ed5-b3ff-1a0580b28eed', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='4e4533e0557f84e65ae698f9e69e5c2d88e2388689ef47dc6d2cd542b6b4dcf2')}, text='进行原生小程序开发\n\n小程序有2种开发方式：\n\n- 基于小程序原生api和组件api实现项目，这是最简单的实现方式\n\t- 直接用原生api，性能最好\n\t- 不利于开发复杂应用\n\t- 不利于复用\n- 使用第三方开发框架，比如 taro 等，\n\t- 最终也会将代码编译为小程序原生形式\n\t- 性能不如直接用原生api\n\t- 利于复用，比如编译为支付宝小程序等\n\t- 利于开发部署过程的自动化', mimetype='text/plain', start_char_idx=2, end_char_idx=195, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.785804939449195)

In [17]:
documents[3]

Document(id_='de3f2c28-3ce1-48fa-aef0-052b96db228a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\ndocker container proxy\n\n参见：https://docs.docker.com/network/proxy/\n\n编辑：`~/.docker/config.json`\n\n```json\n{\n  "proxies": {\n    "default": {\n      "httpProxy": "http://10.1.1.100:8118",\n      "httpsProxy": "http://10.1.1.100:8118",\n      "noProxy": "localhost,*.test.example.com,.example2.com"\n    }\n  }\n}  \n```\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')